# Mask R-CNN - Train modified model on old shapes dataset

### the modified model does not include any mask related heads or losses 



In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
import tensorflow as tf
import keras.backend as KB
import numpy as np
from mrcnn.datagen     import data_generator, load_image_gt
from mrcnn.callbacks   import get_layer_output_1,get_layer_output_2
from mrcnn.utils       import mask_string
import mrcnn.visualize as visualize
from mrcnn.prep_notebook import prep_oldshapes_train, load_model
# model_file = "E:\Models\mrcnn_oldshape_train_logs\TrainMRCNN\mask_rcnn_shapes_1418.h5"
model_file = "E:\Models\mrcnn_oldshape_train_logs\shapes20180606T1150\mask_rcnn_shapes_0008.h5"
model, dataset_train, dataset_val, train_generator, val_generator, config = prep_oldshapes_train(init_with = 'last', FCN_layers = True, batch_sz = 16)

D:\Program Files\Anaconda3\envs\TF_gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


 windows  Windows
Tensorflow Version: 1.6.0   Keras Version : 2.1.4 
 Initialize config object - super
(56, 56)
>>> Initialize model WITHOUT MASKING LAYERS!!!!
    set_log_dir: Checkpoint path set to : E:\models\mrcnn_oldshape_train_logs\shapes20180606T1609\mask_rcnn_shapes_{epoch:04d}.h5
    set_log_dir: self.epoch set to 0 

>>> Resnet Graph 
     Input_image shape : (?, 128, 128, 3)
     After ZeroPadding2D  : (?, 134, 134, 3) (?, 134, 134, 3)
     After Conv2D padding : (?, 64, 64, 64) (?, 64, 64, 64)
     After BatchNorm      : (?, 64, 64, 64) (?, 64, 64, 64)
     After MaxPooling2D   : (?, 32, 32, 64) (?, 32, 32, 64)

>>> Feature Pyramid Network (FPN) Graph 
     FPN P2 shape : (None, 32, 32, 256)
     FPN P3 shape : (None, 16, 16, 256)
     FPN P4 shape : (None, 8, 8, 256)
     FPN P5 shape : (None, 4, 4, 256)
     FPN P6 shape : (None, 2, 2, 256)

>>> RPN Layer 
     Input_feature_map shape : (?, ?, ?, 256)
     anchors_per_location    : 3
     depth                   : 256
   

   FCN fully connected 1 (fcn_fc1) shape is :  (16, 16, 16, 1024)
   FCN fully connected 2 (fcn_fc2) shape is :  (16, 16, 16, 1024)
   FCN final conv2d (fcn_classify) shape is :  (16, 16, 16, 4)  keras_tensor  True
   h_factor :  8.0 w_factor :  8.0

>>> BilinearUpSampling2D layer
     data_format :  channels_last
     size        :  (8.0, 8.0)
     target_size :  None
     input_spec  :  [InputSpec(ndim=4)]
     call resize_images_bilinear with size:  (8.0, 8.0)
     CHANNELS LAST: X:  (16, 16, 16, 4)  KB.int_shape() :  (None, 16, 16, 4)
     target_height   :  None  target_width  :  None
     new_shape (2):  (2,) (2,)
     new_shape (3):  (2,) (2,)
     X after image.resize_bilinear:  (16, ?, ?, 4)
     Dimensions of X after set_shape() :  (16, 128, 128, 4)
     BilinearUpSampling2D. compute_output_shape()
     Bilinear output shape is: None , 128 , 128 , 4
   FCN Bilinear upsmapling layer  shape is :  (16, 128, 128, 4)  Keras tensor  True


    L2 normalization ---------------------

In [ ]:
model.layer_info()

In [ ]:
from mrcnn.prep_notebook import load_model
load_model(model, init_with='last')

## Training

Train in two stages:
1. Only the heads. Here we're freezing all the backbone layers and training only the randomly initialized layers (i.e. the ones that we didn't use pre-trained weights from MS COCO). To train only the head layers, pass `layers='heads'` to the `train()` function.

2. Fine-tune all layers. For this simple example it's not necessary, but we're including it to show the process. Simply pass `layers="all` to train all layers.

### Training FPN, RPN and MRCNN heads using  Keras.model.fit_generator()

print(config.BATCH_SIZE)
print(model.config.BATCH_SIZE)
print(model.config.LEARNING_RATE)
import keras
keras.__version__

In [ ]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.

# model.train(dataset_train, dataset_val, 
#             learning_rate=config.LEARNING_RATE, 
# #             epochs = 69,
#             epochs_to_run =2, 
#             layers='heads')
## Last run prior to FCN training was 3699, last checkpoint was 3892  ...start at 3899

train_layers = [ 'mrcnn', 'fpn','rpn']
loss_names   = [ "rpn_class_loss", "rpn_bbox_loss" , "mrcnn_class_loss", "mrcnn_bbox_loss"]
model.epoch = 1233
model.config.LEARNING_RATE = 1.0e-4
model.config.STEPS_PER_EPOCH = 7

model.train(dataset_train, dataset_val, 
            learning_rate=model.config.LEARNING_RATE, 
            epochs_to_run =3000, 
#             epochs = 25,            
#             batch_size = 0
#             steps_per_epoch = 0 
            layers = train_layers,
            losses = loss_names,
            min_LR = 1.0e-6,
            )

###  Train FCN head layers

In [ ]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.

# model.train(dataset_train, dataset_val, 
#             learning_rate=config.LEARNING_RATE, 
# #             epochs = 69,
#             epochs_to_run =2, 
#             layers='heads')

## Last run prior to FCN training was 3699, last checkpoint was 3892

train_layers = ['fcn']
loss_names   = [  "fcn_norm_loss"]
model.epoch = 1063
model.config.LEARNING_RATE = 1.0e-4
model.config.STEPS_PER_EPOCH = 8 

model.train(dataset_train, dataset_val, 
            learning_rate=model.config.LEARNING_RATE, 
            epochs_to_run = 500, 
#             epochs = 25,            
#             batch_size = 6,
#             steps_per_epoch = 0 
            layers = train_layers,
            losses = loss_names,
            min_LR = 1.0e-9
            )

['fcn']
['(fcn\\_.*)']
layers regex : (fcn\_.*)
 213  fcn_block1_conv1       (Conv2D              )   TRAIN 
 214  fcn_block1_conv2       (Conv2D              )   TRAIN 
 216  fcn_block2_conv1       (Conv2D              )   TRAIN 
 217  fcn_block2_conv2       (Conv2D              )   TRAIN 
 219  fcn_block3_conv1       (Conv2D              )   TRAIN 
 220  fcn_block3_conv2       (Conv2D              )   TRAIN 
 221  fcn_block3_conv3       (Conv2D              )   TRAIN 
 223  fcn_fc1                (Conv2D              )   TRAIN 
 225  fcn_fc2                (Conv2D              )   TRAIN 
 227  fcn_classify           (Conv2D              )   TRAIN 


 Compile Model :
----------------
    losses        :  ['fcn_norm_loss']
    learning rate :  0.0001
    momentum      :  0.9


 Add losses:
----------------
    losses:  ['fcn_norm_loss']
    keras_model.losses           : []
    Loss: fcn_norm_loss  Related Layer is : fcn_norm_loss
      >> Add add loss for  Tensor("fcn_norm_loss/fcn_no

Epoch 1064/1563
8/8 [==============================] - 30s 4s/step - loss: 0.0273 - fcn_norm_loss: 0.0273 - val_loss: 0.0255 - val_fcn_norm_loss: 0.0255

Epoch 01064: val_loss improved from inf to 0.02554073, saving model to E:\models\mrcnn_oldshape_train_logs\shapes20180606T1150\mask_rcnn_shapes_1064.h5
Epoch 1065/1563
8/8 [==============================] - 22s 3s/step - loss: 0.0270 - fcn_norm_loss: 0.0270 - val_loss: 0.0256 - val_fcn_norm_loss: 0.0256

Epoch 01065: val_loss did not improve
Epoch 1066/1563
8/8 [==============================] - 23s 3s/step - loss: 0.0255 - fcn_norm_loss: 0.0255 - val_loss: 0.0261 - val_fcn_norm_loss: 0.0261

Epoch 01066: val_loss did not improve
Epoch 1067/1563
8/8 [==============================] - 23s 3s/step - loss: 0.0283 - fcn_norm_loss: 0.0283 - val_loss: 0.0255 - val_fcn_norm_loss: 0.0255

Epoch 01067: val_loss improved from 0.02554073 to 0.02548689, saving model to E:\models\mrcnn_oldshape_train_logs\shapes20180606T1150\mask_rcnn_shapes_1067.

8/8 [==============================] - 22s 3s/step - loss: 0.0268 - fcn_norm_loss: 0.0268 - val_loss: 0.0258 - val_fcn_norm_loss: 0.0258

Epoch 01105: val_loss did not improve
Epoch 1106/1563
8/8 [==============================] - 22s 3s/step - loss: 0.0277 - fcn_norm_loss: 0.0277 - val_loss: 0.0256 - val_fcn_norm_loss: 0.0256

Epoch 01106: val_loss did not improve
Epoch 1107/1563
8/8 [==============================] - 22s 3s/step - loss: 0.0258 - fcn_norm_loss: 0.0258 - val_loss: 0.0258 - val_fcn_norm_loss: 0.0258

Epoch 01107: val_loss did not improve
Epoch 1108/1563
8/8 [==============================] - 22s 3s/step - loss: 0.0284 - fcn_norm_loss: 0.0284 - val_loss: 0.0256 - val_fcn_norm_loss: 0.0256

Epoch 01108: val_loss did not improve
Epoch 1109/1563
8/8 [==============================] - 22s 3s/step - loss: 0.0283 - fcn_norm_loss: 0.0283 - val_loss: 0.0259 - val_fcn_norm_loss: 0.0259

Epoch 01109: val_loss did not improve
Epoch 1110/1563
8/8 [==============================] - 2

8/8 [==============================] - 25s 3s/step - loss: 0.0263 - fcn_norm_loss: 0.0263 - val_loss: 0.0259 - val_fcn_norm_loss: 0.0259

Epoch 01147: val_loss did not improve
Epoch 1148/1563
8/8 [==============================] - 24s 3s/step - loss: 0.0266 - fcn_norm_loss: 0.0266 - val_loss: 0.0259 - val_fcn_norm_loss: 0.0259

Epoch 01148: val_loss did not improve

Epoch 01148: ReduceLROnPlateau reducing learning rate to 2.9999999242136255e-05.
Epoch 1149/1563
8/8 [==============================] - 24s 3s/step - loss: 0.0274 - fcn_norm_loss: 0.0274 - val_loss: 0.0258 - val_fcn_norm_loss: 0.0258

Epoch 01149: val_loss did not improve
Epoch 1150/1563
8/8 [==============================] - 24s 3s/step - loss: 0.0288 - fcn_norm_loss: 0.0288 - val_loss: 0.0255 - val_fcn_norm_loss: 0.0255

Epoch 01150: val_loss did not improve
Epoch 1151/1563
8/8 [==============================] - 24s 3s/step - loss: 0.0271 - fcn_norm_loss: 0.0271 - val_loss: 0.0254 - val_fcn_norm_loss: 0.0254

Epoch 01151:


Epoch 01188: val_loss did not improve
Epoch 1189/1563
8/8 [==============================] - 24s 3s/step - loss: 0.0271 - fcn_norm_loss: 0.0271 - val_loss: 0.0259 - val_fcn_norm_loss: 0.0259

Epoch 01189: val_loss did not improve
Epoch 1190/1563
8/8 [==============================] - 23s 3s/step - loss: 0.0269 - fcn_norm_loss: 0.0269 - val_loss: 0.0262 - val_fcn_norm_loss: 0.0262

Epoch 01190: val_loss did not improve
Epoch 1191/1563
8/8 [==============================] - 23s 3s/step - loss: 0.0263 - fcn_norm_loss: 0.0263 - val_loss: 0.0254 - val_fcn_norm_loss: 0.0254

Epoch 01191: val_loss did not improve
Epoch 1192/1563
8/8 [==============================] - 24s 3s/step - loss: 0.0260 - fcn_norm_loss: 0.0260 - val_loss: 0.0258 - val_fcn_norm_loss: 0.0258

Epoch 01192: val_loss did not improve
Epoch 1193/1563
8/8 [==============================] - 24s 3s/step - loss: 0.0276 - fcn_norm_loss: 0.0276 - val_loss: 0.0262 - val_fcn_norm_loss: 0.0262

Epoch 01193: val_loss did not improve
E

8/8 [==============================] - 23s 3s/step - loss: 0.0263 - fcn_norm_loss: 0.0263 - val_loss: 0.0258 - val_fcn_norm_loss: 0.0258

Epoch 01231: val_loss did not improve
Epoch 1232/1563
8/8 [==============================] - 23s 3s/step - loss: 0.0263 - fcn_norm_loss: 0.0263 - val_loss: 0.0259 - val_fcn_norm_loss: 0.0259

Epoch 01232: val_loss did not improve
Epoch 1233/1563
8/8 [==============================] - 23s 3s/step - loss: 0.0268 - fcn_norm_loss: 0.0268 - val_loss: 0.0259 - val_fcn_norm_loss: 0.0259

Epoch 01233: val_loss did not improve
Epoch 1234/1563
8/8 [==============================] - 24s 3s/step - loss: 0.0263 - fcn_norm_loss: 0.0263 - val_loss: 0.0258 - val_fcn_norm_loss: 0.0258

Epoch 01234: val_loss did not improve
Epoch 1235/1563
8/8 [==============================] - 23s 3s/step - loss: 0.0271 - fcn_norm_loss: 0.0271 - val_loss: 0.0258 - val_fcn_norm_loss: 0.0258

Epoch 01235: val_loss did not improve
Epoch 1236/1563
8/8 [==============================] - 2


Epoch 01272: val_loss did not improve
Epoch 1273/1563
8/8 [==============================] - 23s 3s/step - loss: 0.0265 - fcn_norm_loss: 0.0265 - val_loss: 0.0256 - val_fcn_norm_loss: 0.0256

Epoch 01273: val_loss did not improve
Epoch 1274/1563
8/8 [==============================] - 23s 3s/step - loss: 0.0274 - fcn_norm_loss: 0.0274 - val_loss: 0.0258 - val_fcn_norm_loss: 0.0258

Epoch 01274: val_loss did not improve
Epoch 1275/1563
8/8 [==============================] - 23s 3s/step - loss: 0.0282 - fcn_norm_loss: 0.0282 - val_loss: 0.0259 - val_fcn_norm_loss: 0.0259

Epoch 01275: val_loss did not improve
Epoch 1276/1563
8/8 [==============================] - 23s 3s/step - loss: 0.0270 - fcn_norm_loss: 0.0270 - val_loss: 0.0261 - val_fcn_norm_loss: 0.0261

Epoch 01276: val_loss did not improve
Epoch 1277/1563
8/8 [==============================] - 23s 3s/step - loss: 0.0275 - fcn_norm_loss: 0.0275 - val_loss: 0.0260 - val_fcn_norm_loss: 0.0260

Epoch 01277: val_loss did not improve
E

8/8 [==============================] - 22s 3s/step - loss: 0.0272 - fcn_norm_loss: 0.0272 - val_loss: 0.0255 - val_fcn_norm_loss: 0.0255

Epoch 01315: val_loss did not improve
Epoch 1316/1563
8/8 [==============================] - 21s 3s/step - loss: 0.0281 - fcn_norm_loss: 0.0281 - val_loss: 0.0257 - val_fcn_norm_loss: 0.0257

Epoch 01316: val_loss did not improve
Epoch 1317/1563
8/8 [==============================] - 21s 3s/step - loss: 0.0267 - fcn_norm_loss: 0.0267 - val_loss: 0.0257 - val_fcn_norm_loss: 0.0257

Epoch 01317: val_loss did not improve
Epoch 1318/1563
8/8 [==============================] - 21s 3s/step - loss: 0.0250 - fcn_norm_loss: 0.0250 - val_loss: 0.0259 - val_fcn_norm_loss: 0.0259

Epoch 01318: val_loss did not improve
Epoch 1319/1563
8/8 [==============================] - 21s 3s/step - loss: 0.0263 - fcn_norm_loss: 0.0263 - val_loss: 0.0256 - val_fcn_norm_loss: 0.0256

Epoch 01319: val_loss did not improve
Epoch 1320/1563
8/8 [==============================] - 2

In [ ]:
model.keras_model.losses
print(model.keras_model.metrics_names)

## - Training heads using train_in_batches ()

We need to use this method for the time being as the fit generator does not have provide EASY access to the output in Keras call backs. By training in batches, we pass a batch through the network, pick up the generated RoI detections and bounding boxes and generate our semantic / gaussian tensors ...


In [ ]:
model.train_in_batches(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE/6, 
            epochs_to_run = 3,
            layers='heads')

## Fine Tuning
Fine tune all layers

In [ ]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=211,
            layers="all")

# train_layers = ['fcn']
# loss_names   = [  "fcn_norm_loss"]
# model.epoch = 208
# model.config.LEARNING_RATE = 1.0e-4
# model.config.STEPS_PER_EPOCH = 8 

# model.train(dataset_train, dataset_val, 
#             learning_rate=model.config.LEARNING_RATE, 
#             epochs_to_run = 500, 
# #             epochs = 25,            
# #             batch_size = 6,
# #             steps_per_epoch = 0 
#             layers = train_layers,
#             losses = loss_names,
#             min_LR = 1.0e-7
#             )

## Save 

In [ ]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
model_path = os.path.join(MODEL_DIR, "mask_rcnn_shapes_post_training.h5")
model.keras_model.save_weights(model_path)

In [ ]:
# model.keras_model.summary(line_length=132, positions=[0.30,0.75, .83, 1. ])

In [ ]:
train_batch_x, train_batch_y = next(train_generator)
imgmeta_idx = model.keras_model.input_names.index('input_image_meta')
img_meta    = train_batch_x[imgmeta_idx]

for img_idx in range(config.BATCH_SIZE):
    image_id = img_meta[img_idx,0]
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    print('Image id: ',image_id)
    print('Image meta', img_meta[img_idx])
    print('Classes (1: circle, 2: square, 3: triangle ): ',class_ids)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

In [ ]:
model.layer_info()

In [ ]:
# train_batch_x, train_batch_y = next(train_generator)

model_output = get_layer_output_2(model.keras_model, train_batch_x, 1)
# model_output = get_layer_output_1(model.keras_model, train_batch_x, [ 26], 1)

print(len(model_output))

# rpn_class_loss            = model_output[0]          # layer: 11   shape: (1, 1)
# rpn_bbox_loss             = model_output[1]          # layer: 12   shape: (1, 1)
# mrcnn_class_loss          = model_output[2]          # layer: 13   shape: (1, 1)
# mrcnn_bbox_loss           = model_output[3]          # layer: 14   shape: (1, 1)
# fcn_normalized_loss       = model_output[0]          # layer: 26   shape: (1, 1)

# print(type(output_rois))
for i in model_output:
    print( i.shape)
# print('FCN Normalized Loss is :', fcn_normalized_loss)    


In [ ]:
input_image      =  train_batch_x[0]
input_image_meta =  train_batch_x[1]
input_rpn_match  =  train_batch_x[2]
input_rpn_bbox   =  train_batch_x[3]
input_gt_class_ids = train_batch_x[4]
input_gt_bboxes    = train_batch_x[5]
input_gt_masks     = train_batch_x[6]
print(' Input image shape is :', input_image.shape)
h, w = input_image.shape[1], input_image.shape[2]      #  tf.shape(input_image)[1], tf.shape(input_image)[2]
input_normlzd_gt_bboxes = tf.identity(input_gt_bboxes / [h,w,h,w])

# gt_masks   =  train_batch_x[6]
print(' input_rpn_match    ', input_rpn_match.shape)
print(' input_rpn_bbox     ', input_rpn_bbox.shape)
print(' input_gt_class_ids ', input_gt_class_ids.shape)
print(' input_gt_bboxes    ', input_gt_bboxes.shape)
print(' input_normlzd_gt_bboxes    ', input_normlzd_gt_bboxes.shape)
